# TensorFlow / Keras

- Dense Network using Keras API
- [KerasTuner](https://keras.io/keras_tuner/) for HyperParameters searching.

&rarr; [Experiment 3 - Keras](#Experiment-3-(Keras))

**Author:** BrenoAV

**Last Date Modified:** 2/4/2024

# Load Dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data.csv", sep="\t", encoding="utf-8")

In [3]:
df

,sentence,target,source
0,So there is no way for me to plug it in here i...,0,amazon
1,"Good case, Excellent value.",1,amazon
2,Great for the jawbone.,1,amazon
3,Tied to charger for conversations lasting more...,0,amazon
4,The mic is great.,1,amazon
...,...,...,...
2743,I think food should have flavor and texture an...,0,yelp
2744,Appetite instantly gone.,0,yelp
2745,Overall I was not impressed and would not go b...,0,yelp
2746,"The whole experience was underwhelming, and I ...",0,yelp


## Split dataset

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df["sentence"], df["target"], test_size=0.2, random_state=123)

In [6]:
X_train.shape, y_train.shape

((2198,), (2198,))

In [7]:
X_test.shape, y_test.shape

((550,), (550,))

# Preprocessing

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

2024-02-04 17:10:33.834979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-04 17:10:34.599959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
NUM_WORDS = 5000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)  # Important be only the train data!!!

In [10]:
import pickle

with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [11]:
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)

In [12]:
tokenizer.texts_to_sequences(["The mic is great."])

[[1, 1213, 5, 20]]

In [13]:
X_train_encoded = pad_sequences(X_train_encoded, maxlen=MAX_LEN, padding="post")
X_test_encoded = pad_sequences(X_test_encoded, maxlen=MAX_LEN, padding="post")

In [14]:
X_train_encoded.shape, X_test_encoded.shape

((2198, 100), (550, 100))

# MLFlow

In [15]:
import mlflow
from mlflow.data.pandas_dataset import PandasDataset

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [16]:
dataset: PandasDataset = mlflow.data.from_pandas(df, source="data.csv")

/home/brenoav/.cache/pypoetry/virtualenvs/nlp-text-sentiment-analysis-QACYZSEo-py3.10/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:150: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(
/home/brenoav/.cache/pypoetry/virtualenvs/nlp-text-sentiment-analysis-QACYZSEo-py3.10/lib/python3.10/site-packages/mlflow/data/digest_utils.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


# Experiment 3 (Keras)

In [17]:
import tensorflow as tf
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [18]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-02-04 17:10:35.726445: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-04 17:10:35.752514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-04 17:10:35.752708: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [19]:
experiment_name = "sentiment_analysis_tf_dense"

experiment_tags = {
    "nlp.framework": "TensorFlow",
    "nlp.encoding": "Tokenizer",
    "nlp.model": "Dense Network",
    "nlp.task": "Sentiment Analysis"
}

mlflow.create_experiment(name=experiment_name,
                         artifact_location="mlartifacts",
                         tags=experiment_tags)

'194346053736584605'

In [20]:
class DenseHyperModel(kt.HyperModel):
    def build(self, hp):
        kernel_regularizer = hp.Choice("kernel_regularizer", ["None", "l2"])

        if kernel_regularizer.lower() == "none":
            kernel_regularizer = None

        model = Sequential(name="DenseNetwork")
        model.add(Embedding(input_dim=len(tokenizer.index_word) + 1, 
                            output_dim=hp.Choice("ouput_dim", [50, 75, 100]), 
                            input_length=MAX_LEN))
        model.add(Flatten())
        for i in range(hp.Int("num_layers", 1, 2)):
            model.add(Dense(units=hp.Choice("units", [16, 32, 64]), 
                            activation=hp.Choice("activation", ["relu", "tanh"]), 
                            kernel_regularizer=kernel_regularizer))
        model.add(Dense(units=1, activation="sigmoid"))

        model.compile(loss="binary_crossentropy", 
                      optimizer=Adam(learning_rate=hp.Choice("lr", [1e-2, 1e-3])),
                      metrics=["accuracy"])

        return model

    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            **kwargs,
        )

In [21]:
mlflow.set_experiment(experiment_name=experiment_name)  # It could be use the ID too

tuner = kt.RandomSearch(
    DenseHyperModel(),
    objective='val_accuracy',
    max_trials=25,
    seed=123,
    overwrite=True,
    directory="keras-tuner",
    project_name="sentiment_analysis"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(x=X_train_encoded, y=y_train, 
             epochs=500,
             validation_data=(X_test_encoded, y_test),
             callbacks=[stop_early])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_encoded, 
                    y_train, 
                    validation_data=(X_test_encoded, y_test),
                    callbacks=[stop_early],
                    epochs=500,
)

y_pred = tf.math.round(model.predict(X_test_encoded))

accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
precision = precision_score(y_true=y_test, y_pred=y_pred)
recall = recall_score(y_true=y_test, y_pred=y_pred)
f1 = f1_score(y_true=y_test, y_pred=y_pred)

metrics = {
    "accuracy": accuracy,
    "precision": precision,
    "recall": recall,
    "f1": f1
}

run_name = "_".join([f"{k}_{v}" for k, v in best_hps.values.items()])

with mlflow.start_run(run_name=run_name):
    # Log the hyperparameters
    mlflow.log_params(best_hps.values)

    # Log the metrics
    mlflow.log_metrics(metrics)
    ## Graph of accuracies and losses
    h = history.history
    for j, (train_loss, train_acc, test_loss, test_acc) in enumerate(zip(h["loss"], h["accuracy"], h["val_loss"], h["val_accuracy"])):
        mlflow.log_metric(key="train_loss", value=train_loss, step=j)
        mlflow.log_metric(key="train_acc", value=train_acc, step=j)
        mlflow.log_metric(key="test_loss", value=test_loss, step=j)
        mlflow.log_metric(key="test_acc", value=test_acc, step=j)

    # Log the dataset
    mlflow.log_input(dataset, context="training")

    # Log the model ! New versions it is mlflow.keras !
    mlflow.tensorflow.log_model(model=model,
                                artifact_path="mlartifacts",
                                input_example=X_train_encoded)

    mlflow.log_artifact("keras-tuner", artifact_path="mlartifacts")
    
    mlflow.log_artifact("tokenizer.pkl", artifact_path="mlartifacts")

Trial 25 Complete [00h 00m 01s]
val_accuracy: 0.7890909314155579

Best val_accuracy So Far: 0.8218181729316711
Total elapsed time: 00h 00m 46s
Epoch 1/500
69/69 [==============================] - 1s 3ms/step - loss: 0.8751 - accuracy: 0.5369 - val_loss: 0.6914 - val_accuracy: 0.5527
Epoch 2/500
69/69 [==============================] - 0s 2ms/step - loss: 0.6290 - accuracy: 0.7944 - val_loss: 0.6226 - val_accuracy: 0.7455
Epoch 3/500
69/69 [==============================] - 0s 2ms/step - loss: 0.4558 - accuracy: 0.9167 - val_loss: 0.5851 - val_accuracy: 0.7745
Epoch 4/500
69/69 [==============================] - 0s 2ms/step - loss: 0.3251 - accuracy: 0.9645 - val_loss: 0.5536 - val_accuracy: 0.8109
Epoch 5/500
69/69 [==============================] - 0s 2ms/step - loss: 0.2506 - accuracy: 0.9809 - val_loss: 0.5532 - val_accuracy: 0.8073
Epoch 6/500
69/69 [==============================] - 0s 2ms/step - loss: 0.1965 - accuracy: 0.9886 - val_loss: 0.5410 - val_accuracy: 0.8073
Epoch 7/500

/home/brenoav/.cache/pypoetry/virtualenvs/nlp-text-sentiment-analysis-QACYZSEo-py3.10/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


INFO:tensorflow:Assets written to: /tmp/tmpvurtidf2/model/data/model/assets


/home/brenoav/.cache/pypoetry/virtualenvs/nlp-text-sentiment-analysis-QACYZSEo-py3.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


This Jupyter Notebook was **created by BrenoAV**. For any inquiries or feedback, please feel free to create an issue on [GitHub](https://github.com/BrenoAV/NLP-Sentiment-Analysis/issues) 📣.